In [0]:
!wget https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/16188500/rumoureval2019.tar.bz2
!tar xf rumoureval2019.tar.bz2

--2019-11-11 14:15:04--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/16188500/rumoureval2019.tar.bz2
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.100.75
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.100.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16512097 (16M) [application/octet-stream]
Saving to: ‘rumoureval2019.tar.bz2’

rumoureval2019.tar. 100%[===================>]  15.75M  10.3MB/s    in 1.5s    

2019-11-11 14:15:07 (10.3 MB/s) - ‘rumoureval2019.tar.bz2’ saved [16512097/16512097]



In [0]:
!unzip rumoureval2019/rumoureval-2019-test-data.zip
!unzip rumoureval2019/rumoureval-2019-training-data.zip

In [0]:
TRAIN_DATA_PREFIX = "rumoureval-2019-training-data"

from tree2branches import tree2branches
def load_true_labels():
    tweet_label_dict = {}
    veracity_label_dict = {}
    path_dev = os.path.join(TRAIN_DATA_PREFIX, "dev-key.json")
    with open(path_dev, 'r') as f:
        dev_key = json.load(f)

    path_train = os.path.join(TRAIN_DATA_PREFIX, "train-key.json")
    with open(path_train, 'r') as f:
        train_key = json.load(f)

    tweet_label_dict['dev'] = dev_key['subtaskaenglish']
    tweet_label_dict['train'] = train_key['subtaskaenglish']

    veracity_label_dict['dev'] = dev_key['subtaskbenglish']
    veracity_label_dict['train'] = train_key['subtaskbenglish']

    return tweet_label_dict, veracity_label_dict

In [0]:
def load_test_data_twitter(set_path ="rumoureval-2019-test-data/twitter-en-test-data"):
    allconv = []
    train_dev_split = {}
    train_dev_split['dev'] = []
    train_dev_split['train'] = []
    train_dev_split['test'] = []
    tweet_data = sorted(os.listdir(set_path))
    newfolds = [i for i in tweet_data if i[0] != '.']
    tweet_data = newfolds # conversation ids, source post id == conversation id
    conversation = {}
    # build conversations for tweet group

    for tweet_topic in tweet_data:
        path = os.path.join(set_path,tweet_topic)
        tweet_topic_data = sorted(os.listdir(path))
        tweet_topic_data = [i for i in tweet_topic_data if i[0] != '.']
        for foldr in tweet_topic_data:
            flag = 0
            conversation['id'] = foldr
            tweets = []
            path_repl = path + '/' + foldr + '/replies'
            files_t = sorted(os.listdir(path_repl))
            newfolds = [i for i in files_t if i[0] != '.']
            files_t = newfolds
            flag = "test"
            if files_t != []:

                # iterate over json reply files
                for repl_file in files_t:
                    with open(os.path.join(path_repl, repl_file)) as f:
                        for line in f:
                            tw = json.loads(line)
                            tw['used'] = 0

                            tw['set'] = flag
                            tweets.append(tw)
                            if tw['text'] is None:
                                print("Tweet has no text", tw['id'])
                conversation['replies'] = tweets

                path_src = path + '/' + foldr + '/source-tweet'
                files_t = sorted(os.listdir(path_src))
                with open(os.path.join(path_src, files_t[0])) as f:
                    for line in f:
                        src = json.loads(line)
                        src['used'] = 0
                        scrcid = src['id_str']
                        src['set'] = flag

                conversation['source'] = src
                if src['text'] is None:
                    print("Tweet has no text", src['id'])
                path_struct = path + '/' + foldr + '/structure.json'
                with open(path_struct) as f:
                    for line in f:
                        struct = json.loads(line)
                if len(struct) > 1:
                    new_struct = {}
                    new_struct[foldr] = struct[foldr]
                    struct = new_struct
                    # Take item from structure if key is same as source tweet id
                conversation['structure'] = struct

                branches = tree2branches(conversation['structure'])
                conversation['branches'] = branches
                train_dev_split[flag].append(conversation.copy())
                allconv.append(conversation.copy())

            # if no replies are present, still add just source
            else:
                flag = 'test'
                path_src = path + '/' + foldr + '/source-tweet'
                files_t = sorted(os.listdir(path_src))
                with open(os.path.join(path_src, files_t[0])) as f:
                    for line in f:
                        src = json.loads(line)
                        src['used'] = 0
                        scrcid = src['id_str']
                        src['set'] = flag

                conversation['source'] = src
                if src['text'] is None:
                    print("Tweet has no text", src['id'])

                path_struct = path + '/' + foldr + '/structure.json'
                with open(path_struct) as f:
                    for line in f:
                        struct = json.loads(line)
                if len(struct) > 1:
                    # print "Structure has more than one root"
                    new_struct = {}
                    new_struct[foldr] = struct[foldr]
                    struct = new_struct
                    # Take item from structure if key is same as source tweet id
                conversation['structure'] = struct
                branches = tree2branches(conversation['structure'])

                conversation['branches'] = branches
                train_dev_split[flag].append(conversation.copy())
                allconv.append(conversation.copy())
                print(type(conversation['branches']))

                print(foldr)

    return train_dev_split

# %%
def load_dataset():
    # Load labels and split for task A and task B
    tweet_label_dict, veracity_label_dict = load_true_labels()
    dev = tweet_label_dict['dev']
    train = tweet_label_dict['train']
    dev_tweets = dev.keys()
    train_tweets = train.keys()
    # Load folds and conversations
    path_to_folds = os.path.join(TRAIN_DATA_PREFIX, 'twitter-english')
    folds = sorted(os.listdir(path_to_folds))
    newfolds = [i for i in folds if i[0] != '.']
    folds = newfolds
    cvfolds = {}
    allconv = []
    train_dev_split = {}
    train_dev_split['dev'] = []
    train_dev_split['train'] = []
    train_dev_split['test'] = []
    # iterate over all tweet groups [reffered to as 'folds'] - charliehebdo etc...
    for nfold, fold in enumerate(folds):
        path_to_tweets = os.path.join(path_to_folds, fold)
        tweet_data = sorted(os.listdir(path_to_tweets))
        newfolds = [i for i in tweet_data if i[0] != '.']
        tweet_data = newfolds # conversation ids, source post id == conversation id
        conversation = {}
        # build conversations for tweet group
        for foldr in tweet_data:
            flag = 0
            conversation['id'] = foldr
            tweets = []
            path_repl = path_to_tweets + '/' + foldr + '/replies'
            files_t = sorted(os.listdir(path_repl))
            newfolds = [i for i in files_t if i[0] != '.']
            files_t = newfolds
            if files_t != []:
                # iterate over json reply files
                for repl_file in files_t:
                    with open(os.path.join(path_repl, repl_file)) as f:
                        for line in f:
                            tw = json.loads(line)
                            tw['used'] = 0
                            replyid = tw['id_str']

                            # check if tweet belongs to dev fold
                            if replyid in dev_tweets:
                                tw['set'] = 'dev'
                                tw['label'] = dev[replyid]
                                #                        train_dev_tweets['dev'].append(tw)
                                if flag == 'train':
                                    print("The tree is split between sets", foldr)
                                flag = 'dev'
                            elif replyid in train_tweets:
                                tw['set'] = 'train'
                                tw['label'] = train[replyid]
                                #                        train_dev_tweets['train'].append(tw)
                                if flag == 'dev':
                                    print("The tree is split between sets", foldr)
                                flag = 'train'
                            else:
                                print("Tweet was not found! ID: ", foldr)
                            tweets.append(tw)
                            if tw['text'] is None:
                                print("Tweet has no text", tw['id'])
                conversation['replies'] = tweets

                path_src = path_to_tweets + '/' + foldr + '/source-tweet'
                files_t = sorted(os.listdir(path_src))
                with open(os.path.join(path_src, files_t[0])) as f:
                    for line in f:
                        src = json.loads(line)
                        src['used'] = 0
                        scrcid = src['id_str']
                        src['set'] = flag
                        src['label'] = tweet_label_dict[flag][scrcid]

                conversation['source'] = src
                conversation['veracity'] = veracity_label_dict[flag][scrcid]
                if src['text'] is None:
                    print("Tweet has no text", src['id'])
                path_struct = path_to_tweets + '/' + foldr + '/structure.json'
                with open(path_struct) as f:
                    for line in f:
                        struct = json.loads(line)
                if len(struct) > 1:
                    # I had to alter the structure of this conversation
                    if foldr == '553480082996879360':
                        new_struct = {}
                        new_struct[foldr] = struct[foldr]
                        new_struct[foldr]['553495625527209985'] = struct['553485679129534464']['553495625527209985']
                        new_struct[foldr]['553495937432432640'] = struct['553490097623269376']['553495937432432640']
                        struct = new_struct
                    else:
                        new_struct = {}
                        new_struct[foldr] = struct[foldr]
                        struct = new_struct
                    # Take item from structure if key is same as source tweet id
                conversation['structure'] = struct

                branches = tree2branches(conversation['structure'])
                conversation['branches'] = branches
                train_dev_split[flag].append(conversation.copy())
                allconv.append(conversation.copy())
            else:
                flag = 'train'
                path_src = path_to_tweets + '/' + foldr + '/source-tweet'
                files_t = sorted(os.listdir(path_src))
                with open(os.path.join(path_src, files_t[0])) as f:
                    for line in f:
                        src = json.loads(line)
                        src['used'] = 0
                        scrcid = src['id_str']
                        src['set'] = flag
                        src['label'] = tweet_label_dict[flag][scrcid]

                conversation['source'] = src
                conversation['veracity'] = veracity_label_dict[flag][scrcid]
                if src['text'] is None:
                    print("Tweet has no text", src['id'])

                path_struct = path_to_tweets + '/' + foldr + '/structure.json'
                with open(path_struct) as f:
                    for line in f:
                        struct = json.loads(line)
                if len(struct) > 1:
                    # print "Structure has more than one root"
                    new_struct = {}
                    new_struct[foldr] = struct[foldr]
                    struct = new_struct
                    # Take item from structure if key is same as source tweet id
                conversation['structure'] = struct
                branches = tree2branches(conversation['structure'])

                conversation['branches'] = branches
                train_dev_split[flag].append(conversation.copy())
                allconv.append(conversation.copy())

                print(foldr)

        cvfolds[fold] = allconv
        allconv = []

    return train_dev_split

In [0]:
import os
import json
from pprint import pprint
x=load_dataset()
print(len(x['train']),len(x['dev']))
import pandas as pd
print(x['train'][0]['source']['text'])
print(x.keys(),x['train'][0].keys(),x['train'][0]['replies'][0].keys())
x_tot=x['train']+x['dev']
print(len(x_tot))
# pprint(x['train'][0])

580323060533764097
297 28
France: 10 people dead after shooting at HQ of satirical weekly newspaper #CharlieHebdo, according to witnesses http://t.co/FkYxGmuS58
dict_keys(['dev', 'train', 'test']) dict_keys(['id', 'replies', 'source', 'veracity', 'structure', 'branches']) dict_keys(['contributors', 'truncated', 'text', 'in_reply_to_status_id', 'id', 'favorite_count', 'source', 'retweeted', 'coordinates', 'entities', 'in_reply_to_screen_name', 'id_str', 'retweet_count', 'in_reply_to_user_id', 'favorited', 'user', 'geo', 'in_reply_to_user_id_str', 'lang', 'created_at', 'in_reply_to_status_id_str', 'place', 'used', 'set', 'label'])
325


In [0]:
def convert(label):
  if label=='comment':
    return 0
  elif label=='deny':
    return 1
  elif label=='query':
    return 2
  else:
    return 3
  

In [0]:
# df = pd.DataFrame(columns=['source','reply', 'label'])
# count1=0
# for i in range(len(x_tot)):
#   for j in range(len(x_tot[i]['replies'])):
#     df.loc[count1] = [x_tot[i]['source']['text'],x_tot[i]['replies'][j]['text'],convert(x_tot[i]['replies'][j]['label'])]
#     count1=count1+1
# print(df.head()) 

In [0]:
def convert_veracity():
  res = []
  for item in df['veracity'].values:
    if item == 'true':
      res.append(1)
    elif item == 'false':
      res.append(0)
    elif item == 'unverified':
      res.append(2)
  return res

In [0]:
df = pd.DataFrame(columns=['source', 'veracity'])
count1=0
for i in range(len(x_tot)):
    df.loc[count1] = [x_tot[i]['source']['text'],(x_tot[i]['veracity'])]
    count1=count1+1
print(df.head())

                                              source veracity
0  France: 10 people dead after shooting at HQ of...     true
1  BREAKING: 10 reportedly shot dead at Paris HQ ...     true
2  BREAKING: At least 10 killed in shooting at Fr...     true
3  Eleven dead in shooting at Paris offices of sa...     true
4  BREAKING Charlie Hebdo latest: 11 dead 10 woun...     true


In [0]:
upvote = []
for i in range(len(x_tot)):
  source_res = []
  for j in range(len(x_tot[i]['replies'])):
    source_res.append(x_tot[i]['replies'][j]['label'])
  upvote.append(source_res)

In [0]:
upvote_ratio = []
for item in upvote:
  support = item.count('support')
  upvote_ratio.append(support/len(item))

In [0]:
final_df= pd.DataFrame(
    {
      'source':df['source'].values,
      'upvote_ratio':upvote_ratio,
     
      'veracity':convert_veracity()
       
    })

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 


--2019-11-11 14:15:24--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   114MB/s    in 3.4s    

2019-11-11 14:15:28 (114 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

--2019-11-11 14:15:29--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/

In [0]:
import modeling
import optimization
import run_classifier
import tokenization


In [0]:
import zipfile
folder = 'model_folder'
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)

In [0]:
def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    print(type(lines))
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line[0]
            text_b=str(line[1])
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    else:
        for line in lines:
            text_a = line[0]
            text_b=str(line[1])
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    return examples

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf


X_train, X_test, y_train, y_test = train_test_split(final_df[['source','upvote_ratio']].values , final_df["veracity"].values, test_size=0.2)
print(X_train.shape)

(260, 2)


In [0]:
# if 'COLAB_TPU_ADDR' not in os.environ:
#   print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
# else:
#   tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
#   print ('TPU address is', tpu_address)

#   with tf.Session(tpu_address) as session:
#     devices = session.list_devices()
    
#   print('TPU devices:')
# print(devices)

In [0]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/output15'
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 4e-5
NUM_TRAIN_EPOCHS = 10
WARMUP_PROPORTION = 0.2
MAX_SEQ_LENGTH = 128
BOARD_SIZE=19
DROP_OUT=0.5
# Model configs
SAVE_CHECKPOINTS_STEPS = 100000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 100000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(3)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver =None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
         num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

<class 'numpy.ndarray'>
INFO:tensorflow:Using config: {'_model_dir': 'model_folder/output15', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7db9b8c160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100000, num_shar

In [0]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 50

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [0]:
import datetime
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

Please wait...
INFO:tensorflow:Writing example 0 of 260
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train
INFO:tensorflow:tokens: [CLS] sydney siege : gun ##man forces hostages to hold up isis flag in window http : / / t . co / 9 ##k ##3 ##xx ##eis ##fa http : / / t . co / cat ##x ##c ##j ##yx ##py [SEP] 0 . 1666 ##66 ##66 ##66 ##66 ##66 ##66 ##6 [SEP]
INFO:tensorflow:input_ids: 101 3994 6859 1024 3282 2386 2749 19323 2000 2907 2039 18301 5210 1999 3332 8299 1024 1013 1013 1056 1012 2522 1013 1023 2243 2509 20348 17580 7011 8299 1024 1013 1013 1056 1012 2522 1013 4937 2595 2278 3501 17275 7685 102 1014 1012 27407 28756 28756 28756 28756 28756 28756 2575 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [0]:
predict_examples = create_examples(X_test, '')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)
len(X_test)
print(result)

<class 'numpy.ndarray'>
INFO:tensorflow:Writing example 0 of 65
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 
INFO:tensorflow:tokens: [CLS] statement from prime minister tony abbott on sydney hostage situation . http : / / t . co / m ##h ##k ##nr ##z ##f ##q ##w ##b [SEP] 0 . 0 [SEP]
INFO:tensorflow:input_ids: 101 4861 2013 3539 2704 4116 14455 2006 3994 13446 3663 1012 8299 1024 1013 1013 1056 1012 2522 1013 1049 2232 2243 16118 2480 2546 4160 2860 2497 102 1014 1012 1014 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 

In [0]:
# with open('confidence_scores.txt', 'w') as f:
#   for pred in result:
#     f.write(str(prediction['probabilities']) + '\n')

{}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encode

NameError: ignored

In [0]:
preds = []


for prediction in result:
#       print(prediction)
      preds.append(np.argmax(prediction['probabilities']))
print(len(preds),len(y_test))

{}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encode

In [0]:
tp=0
for i in range(len(y_test)):
  if y_test[i]==preds[i]:
    tp=tp+1
print("accuracy")
print(tp/len(preds))

print(type(y_test))
print(type(preds))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test.tolist(),preds))

accuracy
0.6615384615384615
<class 'numpy.ndarray'>
<class 'list'>
[[ 8  3  1]
 [ 3 20  8]
 [ 3  4 15]]


In [0]:
print(np.shape(y_test),np.shape(preds))
randlist=[]
for i in range(len(y_test)):
  if y_test[i]==preds[i] and y_test[i]==1:
    randlist.append(i)
print(len(randlist))
    

from sklearn.metrics import accuracy_score ,classification_report
# print(preds)
# print("Accuracy of BERT is:",accuracy_score(y_test,preds))

print(classification_report(y_test.tolist(),preds))


(65,) (65,)
20
              precision    recall  f1-score   support

           0       0.57      0.67      0.62        12
           1       0.74      0.65      0.69        31
           2       0.62      0.68      0.65        22

    accuracy                           0.66        65
   macro avg       0.65      0.66      0.65        65
weighted avg       0.67      0.66      0.66        65



In [0]:
def serving_input_fn():
    label_ids = tf.placeholder(tf.int32, [None], name='label_ids')
    input_ids = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH], name='input_ids')
    input_mask = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH], name='input_mask')
    segment_ids = tf.placeholder(tf.int32, [None, MAX_SEQ_LENGTH], name='segment_ids')
    input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn({
        'label_ids': label_ids,
        'input_ids': input_ids,
        'input_mask': input_mask,
        'segment_ids': segment_ids,
    })()
    return input_fn

In [0]:
export_path = estimator.export_saved_model('test_saved_1',serving_input_fn)

In [0]:
!zip -r model1.zip test_saved_1

In [0]:
export_path

In [0]:
from tensorflow.contrib import predictor

with tf.Session() as sess:
  predict_fn = predictor.from_saved_model(export_path)

In [0]:
x=load_test_data_twitter()


In [0]:
x.keys()

In [0]:
# print(len(x['train']),len(x['dev']))
print(x['test'][0]['source']['text'])
print(x.keys(),x['test'][0].keys(),x['test'][0]['replies'][0].keys())

In [0]:
x_tot = x['test']

In [0]:
df = pd.DataFrame(columns=['source','reply'])
count1=0
for i in range(len(x_tot)):
  for j in range(len(x_tot[i]['replies'])):
    df.loc[count1] = [x_tot[i]['source']['text'],x_tot[i]['replies'][j]['text']]
    count1=count1+1
print(df.head()) 

In [0]:
new_examples = create_examples(df.values, '')

In [0]:
new_features = run_classifier.convert_examples_to_features(
    new_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [0]:
new_pred = []
for feature_example in new_features:
  x = feature_example.__dict__
  new_x = dict((i, np.expand_dims(x[i], axis=0)) for i in x if i not in['is_real_example', 'label_id'])
  new_pred.append(np.argmax(predict_fn(new_x)['probabilities']))

In [0]:
len(new_pred)

In [0]:
print(new_pred)

In [0]:
with open('output.txt', 'w') as f:
  for pred in new_pred:
    f.write(str(pred)+'\n')

In [0]:
tp=0
for i in range(len(y_test)):
  if y_test[i]==new_pred[i]:
    tp=tp+1
print("accuracy")
print(tp/len(new_pred))

print(type(y_test))
print(type(new_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test.tolist(),new_pred))

In [0]:
print(np.shape(y_test),np.shape(new_pred))
randlist=[]
for i in range(len(y_test)):
  if y_test[i]==new_pred[i] and y_test[i]==1:
    randlist.append(i)
print(len(randlist))
    

from sklearn.metrics import accuracy_score ,classification_report
# print(preds)
# print("Accuracy of BERT is:",accuracy_score(y_test,preds))

print(classification_report(y_test.tolist(),new_pred))